In [1]:
!pip install tinygrad numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!python3 -m tinygrad.device

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tinygrad.device' found in sys.modules after import of package 'tinygrad', but prior to execution of 'tinygrad.device'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
  METAL     : FAIL /usr/lib/libobjc.dylib: cannot open shared object file: No such file or directory
  AMD       : FAIL No interface for AMD:0 is available
  NV        : FAIL rm_control returned 58: Invalid structure parameter
* CUDA      : PASS
                 + CUDACompiler (default)
                 + PTXCompiler (CUDA_PTX=1 to make default)
                 + NVCCCompiler (CUDA_NVCC=1 to make default)
  QCOM      : FAIL [Errno 2] No such file or directory: '/dev/kgsl-3d0'
  CL        : FAIL OpenCL Error -1001: Unknown error
  CPU       : PASS
                 + ClangJITCompiler (default)
                 + CPULLVMCompiler (CPU_LLVM=1 to make default)
                 - LVPCompiler: libtinymesa not found (MESA_PATH=/usr/lib). See https://github.

In [1]:
%env DEBUG=2

env: DEBUG=2


In [2]:
from typing import Callable
from tinygrad import Tensor, TinyJit, nn
import numpy as np
import csv
import os

class Model:
  def __init__(self):
    self.layers: list[Callable[[Tensor], Tensor]] = [
      nn.Conv2d(3, 32, 5), Tensor.relu,
      nn.Conv2d(32, 32, 5), Tensor.relu,
      nn.BatchNorm2d(32), Tensor.max_pool2d,
      nn.Conv2d(32, 64, 3), Tensor.relu,
      nn.Conv2d(64, 64, 3), Tensor.relu,
      nn.BatchNorm2d(64), Tensor.max_pool2d,
      lambda x: x.mean(axis=(2,3)), nn.Linear(64, 1)]

  def __call__(self, x:Tensor) -> Tensor: return x.sequential(self.layers)

if __name__ == "__main__":
  Tensor.manual_seed(42)

  X, Y = np.load("data/X.npy"), np.load("data/Y.npy")
    
  with open("permutations.txt", "r") as f: permutations = [list(map(int, line.strip().split())) for line in f.readlines()]

  # to start in the middle
  START = 0
  permutations = permutations[START:]

  with open("results3.csv", "a", newline='') as f:
      writer = csv.writer(f)
      writer.writerow(["epoch", "loss", "MAE", "permutation_index"])

  for i, permutation in enumerate(permutations):
    print(f"Starting training split {START+i}") 
    X_train, X_val = X[permutation[:10]], X[permutation[10:13]]
    Y_train, Y_val = Y[permutation[:10]], Y[permutation[10:13]]
    augmented_images_train, augmented_labels_train = [], []
    
    for j in range(X_train.shape[0]):
      augmented_images_train.extend([
        X_train[j],
        np.flip(X_train[j], axis=2),
        np.flip(X_train[j], axis=1),
        np.flip(X_train[j], axis=(1,2)),
      ])
      augmented_labels_train.extend([Y_train[j]] * 4)
    augmented_images_val, augmented_labels_val = [], []
    for j in range(X_val.shape[0]):
      augmented_images_val.extend([
        X_val[j],
        np.flip(X_val[j], axis=2),
        np.flip(X_val[j], axis=1),
        np.flip(X_val[j], axis=(1,2)),
      ])
      augmented_labels_val.extend([Y_val[j]] * 4)
    X_train = np.stack(augmented_images_train)
    Y_train = np.array(augmented_labels_train)
    X_val = np.stack(augmented_images_val)
    Y_val = np.array(augmented_labels_val)
    
    X_train, Y_train, X_val, Y_val = map(Tensor, [X_train, Y_train, X_val, Y_val])

    model = Model()
    opt = nn.optim.Adam(nn.state.get_parameters(model))

    @TinyJit
    @Tensor.train()
    def train_step() -> Tensor:
      opt.zero_grad()
      samples = Tensor.randint(10, high=X_train.shape[0])
      loss = ((model(X_train[samples]).squeeze() - Y_train[samples]) ** 2).mean().backward()
      return loss.realize(*opt.schedule_step())

    # import timeit
    # print(timeit.repeat(train_step, repeat=10, number=1))
      
    @TinyJit
    # need contiguous because of a tinygrad bug
    def get_val_acc() -> Tensor: return ((model(X_val).contiguous().squeeze()-Y_val)*Y_val.reciprocal()).abs().mean()*100
    # print("starting")
    # print(timeit.repeat(get_val_acc, repeat=1, number=1))
    # print(timeit.repeat(get_val_acc, repeat=1, number=1))
    # print(timeit.repeat(get_val_acc, repeat=1, number=1))
    # print(timeit.repeat(get_val_acc, repeat=1, number=1))
    # print(timeit.repeat(get_val_acc, repeat=1, number=1))
    # print(timeit.repeat(get_val_acc, repeat=1, number=1))
    # print("done")
    # exit(0)


    
    
    test_acc = float('nan')
    for j in range(2000):
      loss = train_step()
      if j%100 != 99: continue
      with open("results3.csv", "a", newline='') as f:
        writer = csv.writer(f)
        test_acc = get_val_acc().item()
        writer.writerow([j, loss.item(), test_acc, START+i])
      os.makedirs(f"models2/perm_{START+i}/", exist_ok=True)
      nn.state.safe_save(nn.state.get_state_dict(model), f"models2/perm_{START+i}/model_epoch_{j}.safetensors")



Starting training split 0
Starting training split 1
Starting training split 2
Starting training split 3
Starting training split 4
Starting training split 5
Starting training split 6
Starting training split 7
Starting training split 8
Starting training split 9
